# Titouan Teyssier - TP Classifieur Baysien

In [1]:
import matplotlib.pyplot as plt
import itertools
# recuperation des donnée
from sklearn import datasets
iris = datasets.load_iris()

# print(iris.data)
# print(iris.target)

## 1. Division de l'échantillon d'apprentissage

In [2]:
import numpy as np
Ciris = np.c_[iris.data.reshape(len(iris.data), -1), iris.target.reshape(len(iris.target), -1)]

np.random.seed(100666001)
np.random.shuffle(Ciris)
shuffledIrisData = Ciris[:, :iris.data.size//len(iris.data)].reshape(iris.data.shape)
shuffledIrisTarget = Ciris[:, iris.data.size//len(iris.data):].reshape(iris.target.shape)


In [3]:
# print(shuffledIrisTarget)
# print(shuffledIrisData)
# print(Ciris)


On divise les données en trois parties:
+ Apprentissage 100 premiers example.
+ Dev 30 suivants.
+ Test 20 derniers.

In [4]:
# ApprData = shuffledIrisData[:100]
class Dataset:
    @classmethod
    def create(cls, debut, fin):
        return cls(shuffledIrisData[debut:fin], shuffledIrisTarget[debut:fin])
    def __init__(self, data, target):
        self._data = data
        self._target = [int(t) for t in target]
        self._size = len(data)
        self._classes = None
        
        
    def size(self):
        return self._size
    
    def data(self):
        return self._data
    
    def target(self):
        return self._target
    
    def classes(self):
        if (self._classes):
            return self._classes
        
        self._classes = []
        for c in self._target:
            if c not in self._classes:
                self._classes.append(c)
        
        return self._classes
    
    

Apprentissage = Dataset.create(0,100)
Dev = Dataset.create(100,130)
Test = Dataset.create(130,150)

Apprentissage.classes()

[0, 1, 2]

## 2. Phase d'apprentissage

In [5]:
# import math
# class Gaussienne:
#     def __init__(self, data):
#         self.mu = sum(data)/len(data)
#         self.sigma = sum( ((xi - self.mu)**2 for xi in data))/len(data)
#         self.const = 1/math.sqrt(2*math.pi*self.sigma)
    
#     def show(self):
#         span = 40 * self.sigma
#         x, y = self.calc(self.mu - span, self.mu + span)
#         plt.plot(x, y)
        
#     def calc(self, mini, maxi, n=500):
#         dx = (maxi-mini)/n
#         x = [mini+i*dx for i in range(n)]
#         y = [self.const * math.exp( -((xi-self.mu)**2 / (2*self.sigma)) ) for xi in x]
#         return x, y
#     def p(self, x):
#         return self.const * math.exp( -((x-self.mu)**2 / (2*self.sigma)))
        
        

# class Categorie:
#     def __init__(self, cat, data, lendataset):
#         self._priori = len(data) / lendataset
#         self.name = iris.target_names[cat]
#         self.cat = cat
        
#         self._data = data
#         self._gaussiennes = None
    
#     def priori(self):
#         return self._priori
    
#     def posteriori(self, x, features):
#         return self.priori() * self.prob_cond(x, features)
    
    
#     def gaussiennes(self):
#         if (self._gaussiennes):
#             return self._gaussiennes
        
#         self._gaussiennes = []
        
#         for param in range(len(self._data[0])):
#             tab = [d[param] for d in self._data]
#             self._gaussiennes.append(Gaussienne(tab))
#         return self._gaussiennes
    
#     def prob_cond(self, x, features=[1,1,1,1]):
#         prob = 1
#         for g, feature, xi in zip(self.gaussiennes(),features, x):
#             if feature:
#                 prob *= g.p(xi)
#         return prob
    
#     def showRepartition(self):
        
#         for g in self.gaussiennes():
#             g.show()
            
#         plt.legend(iris.feature_names)
#         plt.xlabel("cm")
#         plt.ylabel("density")
#         plt.title("Features pour la catégorie " + self.name)
#         plt.xlim(0,10)
#         plt.show()

# class Baysien:
#     def __init__(self, dataset):
#         self._dataset = dataset
#         self._priori = None
#         self._categories = None

    
#     def bycat(self):
#         if (not self._categories):
#             self._categories = []
#             for cls in self._dataset.classes():
#                 data = [d for d,t in zip(self._dataset.data(), self._dataset.target()) if t == cls]
#                 self._categories.append( Categorie(cls, data, self._dataset.size()))
#         return self._categories
#     def dataset(self):
#         return self._dataset
#     def selectFeature(self, dev):
#         possibilities = [[int(c) for c in ('{0:0'+str(len(dev.data()[0]))+ 'b}').format(i)] for i in range(16)]
#         diagMax = 0
#         acceptable = []
#         for features in possibilities[1:]:
#             confMat = self.confusion(features, dev)
#             diag = sum((confMat[i][i] for i in range(len(confMat))))
#             if (diag == diagMax):
#                 acceptable.append(features)
#             elif (diag > diagMax):
#                 diagMax = diag
#                 acceptable = [features]
#         return acceptable
        
        
            
#     def confusion(self, features, dev):
#         mat = []
#         for i in range(len(self.bycat())):
#             row = []
#             for j in range(len(self.bycat())):
#                 row.append(0)
#             mat.append(row)
        
#         for x,y in zip(dev.data(), dev.target()):
#             # calcul des probabilité a posteriori
#             p, guess = max([(cat.posteriori(x, features), cat) for cat in self.bycat()])
#             mat[guess.cat][y] += 1
        
#         return mat
            
                
        

In [6]:
# app = Baysien(Apprentissage)
# for cat in app.bycat():
#     cat.showRepartition()



In [7]:

# for dataset in [Apprentissage, Dev, Test]:
#     feat = app.selectFeature(dataset)
#     for f in feat:
#         print(f, app.confusion(f, Test))
#     print()

In [8]:
# [g.mu for g in app.bycat()[0].gaussiennes()]

# Loi Gaussienne MultiDimensionel

In [9]:
def printMat(m):
    for r in m:
        print(r)
    print()

# fonction pour manipuler des matrice:
def determinant(mat):
    n=len(mat)
    if (n == 1):
        return mat[0][0]
    sign = 1
    det = 0
    for i in range(n):
        det += sign * mat[i][0] * determinant([l[1:] for l in mat[:i]+mat[i+1:]])
        sign *= -1
    return det

# a besoin du determinant
def comatrice(mat):
    com = []
    lsign = 1
    n = len(mat)
    for i in range(n):
        sign = lsign
        line = []
        com.append(line)
        for j in range(n):
            cofactor = sign * determinant([l[:j]+l[j+1:] for l in mat[:i]+mat[i+1:]])
            sign*=-1
            line.append(cofactor)
        lsign*=-1
    return com

def transpose(mat):
    tmat=[]
    for j in range(len(mat[0])):
        line = []
        for i in range(len(mat)):
            line.append(mat[i][j])
        tmat.append(line)
    return tmat

# a besoin du determinant, de transpose et de comatrice
def inverse(mat):
    d = determinant(mat)
    if d == 0:
        return None
    inv = transpose(comatrice(mat))
    for l in inv:
        for i in range(len(l)):
            l[i] /= d
    return inv

# a besoin de transpose pour simplifier
def produit(a, b):
    if (len(a[0]) != len(b)):
        return None
    prod = []
    tb = transpose(b)
    for l in a:
        line = []
        prod.append(line)
        for c in tb:
            line.append(sum((va*vb for va,vb in zip(l,c)) ))
    
    return prod



# supprime la ligne L et la colonne C
def supprimeLC(mat, L, C):
    return [l[:C]+l[C+1:] for l in mat[:L]+mat[L+1:]]


In [10]:
# application d'un masque de caractéristique
def transform_sigma(sigma, mask):
    for i, v in reversed(list(enumerate(mask))):
        if not v:
            sigma = supprimeLC(sigma, i, i)
    return sigma

def transform_individu(ind, mask):
    ind=list(ind)
    for i, v in reversed(list(enumerate(mask))):
        if not v:
            ind = ind[:i]+ind[i+1:]
    return ind

def transform_params(sigma, mu, mask):
    return (transform_sigma(sigma, mask), transform_individu(mu, mask))

        

In [11]:
# calcul de la vraisemblance
import math
def gaussienneMultiD(x, sigma, mu):
    det = determinant(sigma)
    dim = len(x)
    denominateur = ((2 * math.pi)**dim * det)**0.5
    txmu = [[xi-mui for xi,mui in zip(x, mu)]]
    xmu = transpose(txmu)
    inv = inverse(sigma)
    dist2 = produit(produit(txmu, inv), xmu)[0][0]
    return math.exp(-dist2/2) / denominateur

In [12]:
# avec des individus donné on peut calculer la matrice de covariance et l'individu moyen
def calcul_des_parametres(individus):
    n = len(individus)
    features = transpose(individus)
    mus = [sum(feature)/n for feature in features]
    
    features_norm = [[v-mu for v in feature] for feature,mu in zip(features,mus)]
    
    varcovar_almost = produit(features_norm, transpose(features_norm))
    sigma = [[v/n for v in line] for line in varcovar_almost]
    return (sigma, mus)



In [13]:
def creer_generateur_de_classifieur(donner_apprentissage):
    # phase d'apprentissage, on apprend pour chaque classe sa probabilité à priori
    # ainsi que les parametre nécessaire au calcul de la vraissemblance
    
    
    
    individus = donner_apprentissage.data()
    cibles = donner_apprentissage.target()
    classes = donner_apprentissage.classes()
    
    individu_par_classe = [[i for i, classe in zip(individus, cibles) if classe == cls] for cls in classes]
    apriori_par_classe = [len(i)/len(individus) for i in individu_par_classe]
    param_vraisemblance_par_classe = [calcul_des_parametres(individu) for individu in individu_par_classe]
    
    
    
    # phase de développement, on influt sur le calcul de la vraisemblance
    # en choisissant les critères à observé
    def generateur_de_classifieur(features_mask):
        
        custom_params = [transform_params(sigma, mu, features_mask) for sigma, mu in param_vraisemblance_par_classe]
        
        #le classifieur calcul les vraisemblance pour chaque classes
        #puis calcul les proba a posteriori
        #et decide en faveur de la plus haute proba a posteriori
        def classifieur_baysien(x):
            x = transform_individu(x, features_mask)
            vraissemblance_par_classe = [gaussienneMultiD(x,sigma,mu) for sigma,mu in custom_params]
            posteriori = [ap* vr for ap,vr in zip(apriori_par_classe, vraissemblance_par_classe)]
            
            max_posteriori = posteriori[0]
            guess = 0
            for cls, p in enumerate(posteriori):
                if (p > max_posteriori):
                    max_posteriori = p
                    guess = cls
            
            return guess
            
        
        return classifieur_baysien
    
    return generateur_de_classifieur



In [14]:
# creation du generateur de classifieur
generateur = creer_generateur_de_classifieur(Apprentissage)

In [15]:
# creation de tous les masque possible
def createMasks(nFeature):
    return [[c=='1' for c in ('{0:0'+str(nFeature)+'b}').format(i)] for i in range(1,2**nFeature)]
masks = createMasks(4)
masks

[[False, False, False, True],
 [False, False, True, False],
 [False, False, True, True],
 [False, True, False, False],
 [False, True, False, True],
 [False, True, True, False],
 [False, True, True, True],
 [True, False, False, False],
 [True, False, False, True],
 [True, False, True, False],
 [True, False, True, True],
 [True, True, False, False],
 [True, True, False, True],
 [True, True, True, False],
 [True, True, True, True]]

In [16]:
# matrice de confusion
def confusion(classifieur, dataset):
    inds = dataset.data()
    targ = dataset.target()
    conf = [[0 for c in dataset.classes()] for c in dataset.classes()]
    for i,t in zip(inds, targ):
        conf[classifieur(i)][t] += 1
    return conf
    

In [17]:
# fonction d'evaluation d'un classifieur
def evaluate_classifieur(classifieur, dataset):
    lines = confusion(classifieur, dataset)
    cols = transpose(lines)
    nClasse = len(dataset.classes())

    instances_classe_i = [sum(line) for line in lines]
    instances_réelement_i = [sum(col) for col in cols]
    instances_correctement_classe_i = [line[i] for i, line in enumerate(lines)]
    
    precision_i = [oki/cli for oki, cli in zip(instances_correctement_classe_i, instances_classe_i) if cli > 0]
    rappel_i = [oki/rei for oki, rei in zip(instances_correctement_classe_i, instances_réelement_i) if rei > 0]
    n = sum(instances_classe_i)
    nOK = sum(instances_correctement_classe_i)
    nKO = n-nOK
    CER = nKO/n
    rappel = sum(rappel_i) / nClasse
    precision = sum(precision_i) / nClasse
    
    stats = {}
    stats["taux_erreur"] = CER * 100
    stats["taux_erreur_±"] = 196 * (CER*(1-CER)/(n**2))**0.5
    stats["taux_bonne_classification"] = nOK / n * 100
    stats["precision"] = precision * 100
    stats["rappel"] = rappel * 100
    stats["fmesure"] = 2*rappel*precision/(rappel+precision) * 100
    return stats



In [18]:
# creation de tous les classifieur possible et evaluation
classifieurs = [generateur(mask) for mask in masks]

dev_evaluations = [evaluate_classifieur(c, Dev) for c in classifieurs]
for ev, mask in zip(dev_evaluations, masks):
    print(mask)
    print(ev)
    print()

[False, False, False, True]
{'taux_erreur': 66.66666666666666, 'taux_erreur_±': 3.0798428691680737, 'taux_bonne_classification': 33.33333333333333, 'precision': 11.11111111111111, 'rappel': 33.33333333333333, 'fmesure': 16.666666666666664}

[False, False, True, False]
{'taux_erreur': 66.66666666666666, 'taux_erreur_±': 3.0798428691680737, 'taux_bonne_classification': 33.33333333333333, 'precision': 11.11111111111111, 'rappel': 33.33333333333333, 'fmesure': 16.666666666666664}

[False, False, True, True]
{'taux_erreur': 3.3333333333333335, 'taux_erreur_±': 1.1727692246648476, 'taux_bonne_classification': 96.66666666666667, 'precision': 96.29629629629629, 'rappel': 97.22222222222221, 'fmesure': 96.75704412546517}

[False, True, False, False]
{'taux_erreur': 73.33333333333333, 'taux_erreur_±': 2.8891487062651486, 'taux_bonne_classification': 26.666666666666668, 'precision': 8.88888888888889, 'rappel': 33.33333333333333, 'fmesure': 14.035087719298245}

[False, True, False, True]
{'taux_err

In [19]:
# fonction réutilisable qui rend le meilleur mask selon le dataset fournit
def get_best_mask(generateur, development_dataset):
    masks = createMasks(4);
    bestmask = None
    lowest_error_rate = 200
    for mask in masks:
        classifieur = generateur(mask)
        stats = evaluate_classifieur(classifieur, development_dataset)
        err = stats["taux_erreur"]
        if (err < lowest_error_rate):
            lowest_error_rate = err
            bestmask = mask
    
    return bestmask
        

bestmask = get_best_mask(generateur, Dev)
best_model = generateur(bestmask)


In [20]:
def presente_stats(classifieur, data, description, nom):
        
        stats = evaluate_classifieur(classifieur, data)
        conf = confusion(classifieur, data)
        print("Statistique de",description,"(stats sur",nom,"):")
        print("De cette matrice de confusion:")
        printMat(conf)
        print("On obtient les statistiques suivantes:")
        print(stats)
        print()

def evaluate_prediction_generalisation(classifieur, data_app, data_test):
    
    
    
    presente_stats(classifieur, data_app, "prediction", "apprentissage")
    presente_stats(classifieur, data_test, "generalisation", "test")

evaluate_prediction_generalisation(best_model, Apprentissage, Test)
    

Statistique de prediction (stats sur apprentissage ):
De cette matrice de confusion:
[34, 0, 0]
[0, 31, 2]
[0, 1, 32]

On obtient les statistiques suivantes:
{'taux_erreur': 3.0, 'taux_erreur_±': 0.3343509533409468, 'taux_bonne_classification': 97.0, 'precision': 96.96969696969697, 'rappel': 96.99754901960786, 'fmesure': 96.98362099499344}

Statistique de generalisation (stats sur test ):
De cette matrice de confusion:
[5, 0, 0]
[1, 5, 1]
[0, 1, 7]

On obtient les statistiques suivantes:
{'taux_erreur': 15.0, 'taux_erreur_±': 3.499299929985997, 'taux_bonne_classification': 85.0, 'precision': 86.30952380952381, 'rappel': 84.72222222222223, 'fmesure': 85.50850734725445}



In [21]:
def dump_classification(model, dataset, name):
    classification = [(list(x), model(x)) for x in dataset.data()]
    print('{:#^50}'.format(name))
    for l in classification:
        print('#',l)
    print('{:#^50}'.format(''))
    print()


In [22]:
# dump_classification(best_model, Dev, "Dev")
# dump_classification(best_model, Test, "Test")

In [23]:
#######################Dev########################
# ([5.5, 2.4, 3.7, 1.0], 1)
# ([6.5, 2.8, 4.6, 1.5], 1)
# ([7.6, 3.0, 6.6, 2.1], 2)
# ([5.4, 3.9, 1.3, 0.4], 0)
# ([5.0, 3.3, 1.4, 0.2], 0)
# ([4.8, 3.0, 1.4, 0.3], 0)
# ([4.8, 3.1, 1.6, 0.2], 0)
# ([6.2, 3.4, 5.4, 2.3], 2)
# ([6.3, 2.7, 4.9, 1.8], 2)
# ([4.6, 3.4, 1.4, 0.3], 0)
# ([7.3, 2.9, 6.3, 1.8], 2)
# ([6.0, 2.7, 5.1, 1.6], 1)
# ([5.9, 3.0, 5.1, 1.8], 2)
# ([5.5, 2.5, 4.0, 1.3], 1)
# ([5.1, 3.8, 1.9, 0.4], 0)
# ([5.6, 2.7, 4.2, 1.3], 1)
# ([6.7, 3.3, 5.7, 2.1], 2)
# ([6.3, 2.5, 4.9, 1.5], 1)
# ([5.4, 3.0, 4.5, 1.5], 1)
# ([5.5, 2.4, 3.8, 1.1], 1)
# ([5.1, 3.3, 1.7, 0.5], 0)
# ([5.8, 2.7, 3.9, 1.2], 1)
# ([6.1, 3.0, 4.9, 1.8], 2)
# ([5.0, 3.4, 1.5, 0.2], 0)
# ([5.6, 2.9, 3.6, 1.3], 1)
# ([6.6, 2.9, 4.6, 1.3], 1)
# ([6.8, 2.8, 4.8, 1.4], 1)
# ([5.4, 3.7, 1.5, 0.2], 0)
# ([5.0, 3.0, 1.6, 0.2], 0)
# ([7.7, 3.0, 6.1, 2.3], 2)
##################################################

#######################Test#######################
# ([6.1, 2.8, 4.0, 1.3], 1)
# ([6.9, 3.1, 4.9, 1.5], 1)
# ([5.1, 3.7, 1.5, 0.4], 0)
# ([5.0, 3.5, 1.6, 0.6], 1)
# ([6.1, 3.0, 4.6, 1.4], 1)
# ([6.1, 2.9, 4.7, 1.4], 1)
# ([6.7, 3.0, 5.2, 2.3], 2)
# ([6.9, 3.2, 5.7, 2.3], 2)
# ([4.9, 3.6, 1.4, 0.1], 0)
# ([6.4, 2.8, 5.6, 2.2], 2)
# ([4.4, 2.9, 1.4, 0.2], 0)
# ([6.6, 3.0, 4.4, 1.4], 1)
# ([6.0, 3.0, 4.8, 1.8], 2)
# ([5.9, 3.2, 4.8, 1.8], 2)
# ([7.7, 2.6, 6.9, 2.3], 2)
# ([5.5, 4.2, 1.4, 0.2], 0)
# ([6.3, 3.3, 6.0, 2.5], 2)
# ([6.1, 2.6, 5.6, 1.4], 1)
# ([5.1, 3.5, 1.4, 0.2], 0)
# ([5.8, 2.8, 5.1, 2.4], 2)
##################################################

# On Peu maintenant terminer par classer tout le corpus

In [24]:
# on fini par faire réapprendre le modele sur tout le corpus
Corpus = Dataset.create(0,150)
presente_stats(best_model, Corpus, "production", "tout le corpus")


Statistique de production (stats sur tout le corpus ):
De cette matrice de confusion:
[49, 0, 0]
[1, 48, 3]
[0, 2, 47]

On obtient les statistiques suivantes:
{'taux_erreur': 4.0, 'taux_erreur_±': 0.25605332777893486, 'taux_bonne_classification': 96.0, 'precision': 96.07535321821035, 'rappel': 96.0, 'fmesure': 96.0376618281679}



# matrice de confusion de mon meilleur modele:
[49, 0, 0]

[**1**, 48, **3**]  Classement de 3 Virginica en Versicolor et de une Setosa en Versicolor

[0, **2**, 47]  Classement de 2 Versicolor en Virginica

In [25]:
print(iris.feature_names)
print(iris.target_names)

# On obtient les statistiques suivantes:
# {'taux_erreur': 3.3333333333333335, 'taux_erreur_±': 0.2345538449329695, 'taux_bonne_classification': 96.66666666666667, 'precision': 96.67867146858742, 'rappel': 96.66666666666667, 'fmesure': 96.6726686949383}

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
['setosa' 'versicolor' 'virginica']


### Par curiosité, on peut aussi regarder sur notre modele final quel masque aurait les meilleurs performance
(en prediction !)

In [26]:
gene = creer_generateur_de_classifieur(Corpus)
curiosity_bestmask = get_best_mask(gene, Corpus)
alter_ego = gene(curiosity_bestmask)
presente_stats(alter_ego, Corpus, "Triche...", "tout le corpus") # -40% d'erreur ! (-2 erreur)

Statistique de Triche... (stats sur tout le corpus ):
De cette matrice de confusion:
[50, 0, 0]
[0, 49, 2]
[0, 1, 48]

On obtient les statistiques suivantes:
{'taux_erreur': 2.0, 'taux_erreur_±': 0.18293333333333334, 'taux_bonne_classification': 98.0, 'precision': 98.01253834867282, 'rappel': 98.0, 'fmesure': 98.00626877331567}

